**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri May  5 22:51:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 56.jpg					       IMG_20211008_163632.jpg
'automatic taxi driver.pdf'		       IMG_20220104_015820.jpg
 B612_20211007_012146_656.jpg		       IMG_20220104_082804.jpg
 B612_20220121_232601_716.jpg		      'ML Projects'
 B612_20220121_234659_596.jpg		      'My Drive'
 B612_20220121_234732_788.jpg		      'New folder'
 Capture.PNG				       Photos
'codeforces-phase-1-4 (1).gsheet'	      'QC (1).pdf'
'codeforces-phase-1-4 (2).gsheet'	       QC.pdf
 codeforces-phase-1-4.gsheet		       Resume.gdoc
 codeforces-phase-1-4.xlsx		       Resume-Hend-Emad.pdf
'Colab Notebooks'			       Schedule.gsheet
'coursera FA..gdoc'			      'study in University'
'C Programming Exam (Responses).gsheet'        uni.gdoc
'data certificate.gdoc'			       Untitled0.ipynb
'Data Science'				       Untitled1.ipynb
'DSP 1.rar'				       Untitled2.ipynb
'Form Access Log: C Programming Exam.gshee

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15514, done.
remote: Total 15514 (delta 0), reused 0 (delta 0), pack-reused 15514
Receiving objects: 100% (15514/15514), 14.17 MiB | 16.04 MiB/s, done.
Resolving deltas: 100% (10412/10412), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |             if (iteration_

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "qrcode" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-05-05 22:53:01--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  18.3MB/s    in 9.4s    

2023-05-05 22:53:12 (16.4 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/what-are-QR-codes-FI2028129.txt  
  inflating: data/obj/0b751b58-2559-42e3-bd5e-cf5eea1b38fa_QR-code-back-print-shirt-example.jpg  
  inflating: data/obj/3_What-is-a-QR-code-Know-everything-about-it-800x500.jpg  
  inflating: data/obj/5d6fe4d3-8138-4f0d-ac13-129622829c6c_QR-code-t-shirt-design-primary-element.jpg  
  inflating: data/obj/5f4e589f415a58145f1c74b7_SproutQR_CTA001.jpg  
  inflating: data/obj/5f5fa831cc78451ee74ef606_SproutQR-Code-Template.jpg  
  inflating: data/obj/7e36c6e6-5036-4caa-b51e-ffc754901646_QR-code-front-print-shirt-example.jpg  
  inflating: data/obj/19_2022-638041984424850944-485.jpg  
  inflating: data/obj/61gNp4xjjTL._AC_UL1500_.jpg  
  inflating: data/obj/71oVnSDOwtL._UL1500_.jpg  
  inflating: data/obj/100x60-full-colour-website002_1.jpg  
  inflating: data/obj/300px-Ltwstampsbl2.jpg  
  inflating: data/obj/611oo7aHntL._AC_UL1500_.jpg  
  inflating: data/obj/4790MTSHIRTTOBIFS.jpg  
  inflating: da

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.502381 0.424603 0.334921 0.338095

['0', '0.502381', '0.424603', '0.334921', '0.338095']
0 data/obj/newtshirtback 45.txt
0 0.502381 0.424603 0.334921 0.338095
0 0.503429 0.500688 0.823045 0.822558

['0', '0.503429', '0.500688', '0.823045', '0.822558']
1 data/obj/qrcode 25.txt
0 0.503429 0.500688 0.823045 0.822558
0 0.497619 0.426190 0.331746 0.341270

['0', '0.497619', '0.426190', '0.331746', '0.341270']
2 data/obj/newtshirtback 21.txt
0 0.497619 0.426190 0.331746 0.341270
0 0.653386 0.317764 0.159363 0.117917

['0', '0.653386', '0.317764', '0.159363', '0.117917']
3 data/obj/customizable-men-s-t-shirts-with-scannable-qr-code.txt
0 0.653386 0.317764 0.159363 0.117917
0 0.499000 0.433500 0.354000 0.353000

['0', '0.499000', '0.433500', '0.354000', '0.353000']
4 data/obj/qr-code-tshirt_1_naturaltshirt 29.txt
0 0.499000 0.433500 0.354000 0.353000
0 0.497685 0.463519 0.902778 0.849785

['0', '0.497685', '0.463519', '0.902778', '0.849785']
5 data/obj/images259.txt
0 0.497685 0.463519 0.9

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/newtshirtback 46.jpg', 'data/obj/qrcode 17.jpg', 'data/obj/ng-coding-unisex-graphic-t-shirt-844122_4e8c3ac6-1e2c-470c-a557-71da4c5f0c27 3.jpg', 'data/obj/5f5fa831cc78451ee74ef606_SproutQR-Code-Template.jpg', 'data/obj/qr-code-tshirt_1_naturaltshirt 50.jpg', 'data/obj/100x60-full-colour-website002_1.jpg', 'data/obj/images320.jpg', 'data/obj/images200.jpg', 'data/obj/qr-code-tshirt_1_naturaltshirt 32.jpg', 'data/obj/ng-coding-unisex-graphic-t-shirt-844122_4e8c3ac6-1e2c-470c-a557-71da4c5f0c27 11.jpg', 'data/obj/ng-coding-unisex-graphic-t-shirt-844122_4e8c3ac6-1e2c-470c-a557-71da4c5f0c27 30.jpg', 'data/obj/ng-coding-unisex-graphic-t-shirt-844122_4e8c3ac6-1e2c-470c-a557-71da4c5f0c27 16.jpg', 'data/obj/c3826d6cdedbffdeabc5b96a26ab46b0.jpg', 'data/obj/qr-code-sample-icon-with-text-hello-vector-26810014.jpg', 'data/obj/qrcode 44.jpg', 'data/obj/qr-code-tshirt_1_naturaltshirt 47.jpg', 'data/obj/qr-code-tshirt_1_naturaltshirt 17.jpg', 'data/obj/ng-coding-unisex-graphic-t-shirt-844122_

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.927308), count: 3, class_loss = 0.000820, iou_loss = 0.023328, total_loss = 0.024148 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.816083), count: 1, class_loss = 0.015775, iou_loss = 0.018592, total_loss = 0.034367 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 167744, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.877992), count: 4, class_loss = 0.000142, iou_loss = 0.029122, total_loss = 0.029264 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.0